In [283]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates


from QUANTAXIS.QAUtil import DATABASE
from QUANTAXIS.QAUtil import  trade_date_sse
from QUANTAXIS.QAUtil.QADate_trade import (
    QA_util_get_pre_trade_date,
    QA_util_get_next_trade_date,
    QA_util_if_tradetime
)

import ind.alpha191 as a191
from scipy import stats
from sklearn import preprocessing
import statsmodels.api as sm



%load_ext autoreload
%autoreload 2
%aimport tools.Cacher


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [357]:
def pretreate_data(data):
    returns = smpl.get_current_return(data,'close')
    returns.name = 'returns'
    ret_forward = smpl.get_forward_return(data,'close')
    ret_forward.name = 'ret_forward'
    # {'Open', 'cap', 'close', 'high', 'ind', 'low', 'returns', 'volume', 'vwap'}
    data = pd.concat([data, returns, ret_forward], axis=1)
    data = data.assign(vwap=data.amount/(data.volume*100))
    data = smpl.add_marketvalue_industry(data)
    data.rename(columns = {"open":"Open",'totalCapital':'cap','industry':'ind'}, inplace=True)
    
    data.rename(columns = {"open":"Open"}, inplace=True)
    data = smpl.add_report_inds(data,'netAssetsPerShare')
    data['pb']=data['close']/data['netAssetsPerShare']
    return data
    # return pd.concat([data, close_ind, vwap_ind, low_ind, high_ind, volume_ind, adv20_ind, adv40_ind, adv81_ind, co_mixed_ind, oh_mixed_ind, lv_mixed_ind], axis=1)




In [16]:
def assemble_stocks_by_codes(codes):
    files = list(map(lambda x:x+'_train_qfq',l))
    return pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),files))).sort_index(level=0)

l = smpl.get_codes_from_blockname('沪深300', sse='all')
df_all =  assemble_stocks_by_codes(l)

In [358]:
df_treated = pretreate_data(df_all)

In [20]:
# ids = [11,24,38,41,42,47,57,69,80,82,83,88,93,97]
ids = [11,24]

# ids = np.arange(1,101)
# np.random.shuffle(ids)

# def generate_alpha_factors(fun_ids, df=None):
#     import ind.alpha191 as a191
#     from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
#     from base.JuUnits import excute_for_multidates

# for i in fun_ids:
for i in ids:
    fun_name = 'alpha'+ '0'*(3- int(np.log10(i)+1)) +str(i)
    # indx = excute_for_multidates(df, lambda x: getattr(a191, fun_name)(x) ,level=1)
    indx = excute_for_multidates(df_treated, lambda x: getattr(a191, fun_name)(x) ,level=1)
    indx.name = fun_name
    # save_cache('{}_train'.format(fun_name), indx, cache_type=CACHE_TYPE.FACTOR)
    print(indx)
# worker=6
# task = task_chunk_split(ids, worker)
# results = parallal_task(worker, generate_alpha_factors, task, df=df_treated)


code
000001   -4.506804e+07
000002    1.596329e+07
000063   -3.630724e+07
000066    3.796777e+06
000069    2.500154e+07
              ...     
603882   -1.483158e+05
603899   -2.834211e+06
603986    1.431323e+05
603993   -1.358729e+08
605499   -4.952623e+05
Name: alpha011, Length: 248, dtype: float64
code
000001   -0.406365
000002   -0.211400
000063    0.276363
000066    0.133742
000069    0.348468
            ...   
603882    1.635349
603899    1.838936
603986    0.895157
603993   -0.114700
605499   -1.127755
Name: alpha024, Length: 248, dtype: float64


In [922]:
# excute_for_multidates(df_treated.sort_index(), lambda x: a191.alpha011(x) ,level=1)
def STD(data, windows):
    return data.rolling(window=windows, min_periods=windows).std()
def MEAN(data, windows):
    return data.rolling(window=windows, min_periods=windows).mean()
def DELTA(data, windows):
    return data.diff(4)
def SEQUENCE(n):
    return np.arange(1,n+1)

def SMA(data,windows,alpha):
    return data.ewm(adjust=False, alpha=float(alpha)/windows, min_periods=windows, ignore_na=False).mean()

def REGBETA(xs, y, n):
    assert len(y)>=n,  'len(y)!>=n !!!'
    def reg(X,Y):
        try:
            if len(Y)>len(X):
                res = sm.OLS(Y[X.index], X).fit().params
            else:
                res = sm.OLS(Y, X).fit().params
        except Exception as e:
            print(e)
            return np.nan
        return res
    return xs.rolling(window=n, min_periods=n).apply(lambda x:reg(x,y))

def COVIANCE(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].cov(B.iloc[x]))
    return se

def CORR(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].corr(B.iloc[x]))
    return se


def alpha170(data, dependencies=['close','volume','high', 'amount'], max_window=20):
    # ((RANK(1/CLOSE)*VOLUME)/MEAN(VOLUME,20))*(HIGH*RANK(HIGH-CLOSE)/(SUM(HIGH,5)/5))-RANK(VWAP-DELAY(VWAP,5))
    # vwap = data['amount'] / (data['volume']*100)
    vwap = data['vwap']
    part1 = (1.0/data['close']).rank(axis=0,pct=True) * data['volume'] / (data['volume'].rolling(window=20,min_periods=20).mean())
    part2 = ((data['high']-data['close']).rank(axis=0,pct=True) * data['high']) / (data['high'].rolling(window=5,min_periods=5).sum()/5.0)
    part3 = (vwap.diff(5)).rank(axis=0,pct=True)
    return (part1*part2-part3)

print(alpha170(df_treated.loc[(slice(None),['000001']),:]).to_list())


[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, -0.4541864195427059, -0.4739890235162801, -0.42850290150695175, -0.4233295277516305, -0.4284885750381574, -0.5135986132613023, -0.18674474144699382, -0.4878408172070382, -0.2894533992566548, -0.47843230230886274, -0.4059900784802012, -0.5257753308831795, -0.41063455763831125, -0.44392570562978945, -0.23431219860051258, -0.33778248615965106, -0.31977098946383875, -0.15451264981700097, -0.3297368954698377, -0.42888354916691535, -0.4173452683432198, -0.36423399782918076, -0.2726933300502671, -0.4360708723132873, -0.4202401902364977, -0.4262666908809628, -0.4772631498470278, -0.5333547491325742, 1.0132287638366226, -0.07666440425688381, -0.26313256052307443, -0.05691795751510076, -0.1554420431116249, -0.10011674118240566, -0.06685877617957314, -0.24525728084812096, -0.2778446654357365, -0.21558663657675942, -0.39499483016082365, -0.4942105832929513, -0.476852234941701, -0.4864940539551391, -0.51

In [885]:
# df_all.loc[(slice('2017-01-10'),['000001','000002','000063']),:]
# pd.concat(indxes,axis=1).sort_index().loc[(slice('2017-01-15'),['000001','000002','000063']),:]

# # a = load_cache('alpha11_train', cache_type=CACHE_TYPE.FACTOR)
# # b = load_cache('alpha24_train', cache_type=CACHE_TYPE.FACTOR)
# pd.concat([a,b],axis=1).sort_index().loc[(slice('2017-01-20'),['000001','000002']),:]
# pd.Series(-1,index=df_treated.loc[(slice(None),['000001']),:].index,dtype=np.dtype('int8'))

# a=alpha004(df_treated.loc[(slice(None),['000001']),:])
# a.rolling(window=5, min_periods=5).corr(a).to_list()
# a.to_list()
# a.rolling(window=5, min_periods=5).apply(lambda x:x.corr(a[x.index])).to_list()
# a[0:50].rolling(window=5, min_periods=5).apply(lambda x:print(x.corr(a[x.index])))


# def alpha083(data, dependencies=['high', 'volume'], max_window=5):
#     # (-1*RANK(COVIANCE(RANK(HIGH),RANK(VOLUME),5)))
#     alpha = COVIANCE(data['high'].rank(pct=True),data['volume'].rank(pct=True),5)*-1
#     return alpha

# print(alpha083(df_treated.loc[(slice(None),['000001']),:]).to_list()[0:100])

# import matplotlib.pyplot as plt
# plt.plot(np.arange(100),alpha083(df_treated.loc[(slice(None),['000001']),:]).to_list()[0:100])
# plt.show()
# plt.plot(np.arange(100),alpha090(df_treated.loc[(slice(None),['000001']),:]).to_list()[0:100])
# plt.show()
(16.616927+16.798685+17.023836+17.345596)/4

16.946261